### dependency 설치

In [ ]:
%pip install -q llama-index
%pip install pypdf
%pip install llama-index-llms-gemini
%pip install -q llama-index google-generativeai
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# llama index를 사용해 gemini api 불러오기

### pdf 불러오기

In [ ]:
from llama_index.core import SimpleDirectoryReader

doc1 = SimpleDirectoryReader(input_files=["data/항만법.pdf"]).load_data()

f:\study\llamaIndex\gemini\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hedge3x\AppData\Local\llama_index\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Error while downloading from https://cdn-lfs-us-1.huggingface.co/repos/23/2c/232ca60237b0bb19bb6c28c5a6c8af79f2e423333a96

ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'cdn-lfs-us-1.huggingface.co\', port=443): Max retries exceeded with url: /repos/23/2c/232ca60237b0bb19bb6c28c5a6c8af79f2e423333a9626aad445543b80fbf31e/b5e0ce3470abf5ef3831aa1bd5553b486803e83251590ab7ff35a117cf6aad38?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1724498168&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNDQ5ODE2OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzIzLzJjLzIzMmNhNjAyMzdiMGJiMTliYjZjMjhjNWE2YzhhZjc5ZjJlNDIzMzMzYTk2MjZhYWQ0NDU1NDNiODBmYmYzMWUvYjVlMGNlMzQ3MGFiZjVlZjM4MzFhYTFiZDU1NTNiNDg2ODAzZTgzMjUxNTkwYWI3ZmYzNWExMTdjZjZhYWQzOD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9uc2UtY29udGVudC10eXBlPSoifV19&Signature=rOcD~ORJklyaDEmAU1CBDLJ~XPZbdln2-R9OIGxmaynGX29Ugbe6MV4ydV~e0pDFLT7NHjqPc7VWDxpKmi~tieZCE

### 임베딩 다운로드

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model_ko = HuggingFaceEmbedding(model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr") 

### llama index 설정

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.llms.gemini import Gemini
from llama_index.core import ServiceContext

llm = Gemini(model_name='models/gemini-1.5-flash', request_timeout=120.0)

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=800, chunk_overlap=20, embed_model=embed_model_ko)
index = VectorStoreIndex.from_documents(doc1,service_context=service_context,show_progress=True)

index.storage_context.persist()

query_engine = index.as_query_engine()


C:\Users\hedge3x\AppData\Local\Temp\ipykernel_12996\517383727.py:16: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, chunk_size=800, chunk_overlap=20, embed_model=embed_model_ko)
Generating embeddings:   0%|          | 0/107 [00:00<?, ?it/s]

KeyboardInterrupt: 

### 질문과 답변 출력

In [ ]:
response = query_engine.query("항만에 대해 정확한 정의를 설명해봐")

print(response)

항만은 선박의 출입, 사람의 승선ㆍ하선, 화물의 하역ㆍ보관 및 처리, 해양친수활동 등을 위한 시설과 화물의 조립ㆍ가공ㆍ포장ㆍ제조 등 부가가치 창출을 위한 시설이 갖추어진 곳을 말합니다. 

